In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!pip install tf2cv tensorflow>=2.0.0

In [0]:
from tensorflow.python.client import device_lib
dev=device_lib.list_local_devices()
dev[len(dev)-1]

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13932847434262680084
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"

In [0]:
#global consts
IMG_SIZE='137x236'
HEIGHT = 137
WIDTH = 236
BATCH_SIZE=128
DROP_OUT=0.0
FOLD=0
TRAIN_DIR = './train/'
TRAIN_IMG_DIR=TRAIN_DIR+str(IMG_SIZE)
EPOCHS = 200
STATS_FS = (0.055029564364430086, 0.17228061284674265)
STATS_128 = (0.0692, 0.2051)
EXPERIMENT='sernext_3cmc_fullsize_augrot'
INPUT_DIR ='/content/drive/My Drive/kaggle/bengali/input/'
OUTPUT_DIR = '/content/drive/My Drive/kaggle/bengali/output/'
IMPORT_DIR = '/content/drive/My Drive/kaggle/bengali/nbs/'
EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+'/'
CHKP_DIR = EXPERIMENT_DIR+'chkp/'
OUTPUT_SUBDIR = ''
DATASET_224='224x224-bengali.zip'
DATASET_128='grapheme-imgs-128x128.zip'
DATASET_FS = 'bengali-full-size.zip'
TRAIN_WITH_FOLD = 'train_with_fold_20.csv'
PRETRAINED = False
PRETRAINED_WEIGHTS = ''
LOG_FILE = EXPERIMENT_DIR+'log_'+EXPERIMENT+'.csv'
#INITIAL_EPOCH=40
#HEAD_WD=1e-3
#BACKBONE_WD = 1e-4

if IMG_SIZE == 128: DATASET=DATASET_128
if IMG_SIZE == 224: DATASET=DATASET_224
if IMG_SIZE == '137x236' : DATASET=DATASET_FS


#lr sheduler consts
LR_MAX = 0.001
LR_MIN = 0.00001
T_MAX = 100

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append(IMPORT_DIR+'py/models/')
sys.path.append(IMPORT_DIR+'py/utils/')
sys.path.append(IMPORT_DIR+'py/callbacks/')
sys.path.append(IMPORT_DIR+'py/opts/')

In [0]:
#import local modules
#from mish import Mish,mish
from generators import *
from kaggle_metric import *
#from cosine import CosineAnnealingScheduler
from kagglevalidation import KaggleValidationFS
#from lookahead import Lookahead
from se_resnext50 import build_se_resnext50_model_fs_wod
#from md121v2 import build_md121_v2_model
#from losses import categorical_focal_loss


#import external modules
import numpy as np 
import pandas as pd 
import random

import os
import errno

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

In [0]:
try:
  os.mkdir(TRAIN_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_DIR+' already exists')
    else:
        raise
try:
  os.mkdir(TRAIN_IMG_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_IMG_DIR+' already exists')
    else:
        raise

try:
  os.mkdir(EXPERIMENT_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(EXPERIMENT_DIR+' already exists')
    else:
        raise

try:
  os.mkdir(CHKP_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(CHKP_DIR+' already exists')
    else:
        raise


os.system('cp '+ '"'+INPUT_DIR+DATASET+'" ' + TRAIN_DIR)
os.system('cp '+ '"'+INPUT_DIR+TRAIN_WITH_FOLD+'" ' + TRAIN_DIR)

/content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_augrot/ already exists
/content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_augrot/chkp/ already exists


0

In [0]:
if PRETRAINED: os.system('cp '+ '"'+OUTPUT_DIR+OUTPUT_SUBDIR+PRETRAINED_WEIGHTS+'" '+TRAIN_DIR)

In [0]:
os.system('unzip -q '+TRAIN_DIR+DATASET+ ' -d '+ TRAIN_IMG_DIR)

0

In [0]:
!ls train

137x236  bengali-full-size.zip	train_with_fold_20.csv


In [0]:
dataset_np = pd.read_csv(TRAIN_DIR+TRAIN_WITH_FOLD).to_numpy()
dataset_np[:,0]+='.png'
dataset_m = dataset_np.shape[0]


valid_m = np.where(dataset_np[:,6]==FOLD)[0].shape[0]
train_m = dataset_m-valid_m

fold_train_inds = np.where(dataset_np[:,6] != FOLD)
fold_valid_inds = np.where(dataset_np[:,6] == FOLD)

train_np = dataset_np[fold_train_inds]
valid_np = dataset_np[fold_valid_inds]

train_df = pd.DataFrame(train_np)
valid_df = pd.DataFrame(valid_np)

train_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)
valid_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)

train_df.drop(columns=[5,6], inplace=True)
valid_df.drop(columns=[5,6], inplace=True)

In [0]:
cons_unique=np.unique(train_np[:,3])
cons_y_train = train_np[:,3]
vowel_unique=np.unique(train_np[:,2])
vowel_y_train = train_np[:,2]
root_unique =np.unique(train_np[:,1])
root_y_train = train_np[:,1]

cons_class_weights = class_weight.compute_class_weight('balanced',
                                                 cons_unique,
                                                 cons_y_train)

vowel_class_weights = class_weight.compute_class_weight('balanced',
                                                 vowel_unique,
                                                 vowel_y_train)

root_class_weights = class_weight.compute_class_weight('balanced',
                                                 root_unique,
                                                 root_y_train)
cons_cw_dict=dict(enumerate(cons_class_weights))
vowel_cw_dict=dict(enumerate(vowel_class_weights))
root_cw_dict=dict(enumerate(root_class_weights))

model_cw={}
model_cw['root']=root_cw_dict
model_cw['vowel']=vowel_cw_dict
model_cw['consonant']=cons_cw_dict

In [0]:
model =  build_se_resnext50_model_fs_wod(height=HEIGHT, width=WIDTH)

In [0]:
if 0: model.summary()

In [0]:
if 0:
  from tensorflow.keras.utils import plot_model
  plot_model(model, to_file=OUTPUT_DIR+'model3.png',show_shapes=True)

In [0]:
if 0:
  for (n, layer) in enumerate(model.layers[1].layers):
    if 'activation' in layer.get_config() and layer.get_config()['activation'] == 'relu':
      print('replacing #{}: {}, {}'.format(n, layer, layer.activation))
      layer.activation = Mish(mish)
      print('-> {}'.format(layer.activation))

In [0]:
TRAIN_DIR+PRETRAINED_WEIGHTS

'./train/'

In [0]:
if 0: 
  model.load_weights(TRAIN_DIR+PRETRAINED_WEIGHTS)
  valid_gen = test_batch_generator(valid_df,TRAIN_IMG_DIR,batch_size=512)
  root_predicts,vowel_predicts,cons_predicts=get_p_dicts(model,valid_gen)
  root_recalls,_,_ = compute_cm(valid_df,root_predicts,vowel_predicts,cons_predicts)
  root_weights = 1./np.power(root_recalls,2)
else:
  root_weights = 1.0


In [0]:
if 0: root_weights[62]

In [0]:
cutout_datagen = ImageDataGenerator(rotation_range = 15,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   preprocessing_function=get_random_eraser(s_l=0.05,s_h=0.1,r_1=0.8,r_2=1.2,v_l=0,v_h=0))

mcm_datagen = ImageDataGenerator(rotation_range = 15,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1)

In [0]:
columns=["root_class","vowel_class", "cons_class"]

mcm_generator1 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")

mcm_generator2 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")

mcm_generator3 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")


cutout_generator = cutout_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")


Found 190798 validated image filenames.
Found 190798 validated image filenames.
Found 190798 validated image filenames.
Found 190798 validated image filenames.


In [0]:
def ps_batch(batch, n=4, p=0.5, s_l=0.05,s_h=0.1,r_1=0.8,r_2=1.2, st=STATS_FS):
        x,y = prep_batch_fs(batch,st=st)
        p_1 = np.random.rand()

        if p_1 > p:
            return x,y

        bs,img_h, img_w, img_c = x.shape

        for i in range(bs):
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / n*r))
            h = int(np.sqrt(s * r/n))
            left = np.random.randint(0, img_w-w-1,size=n)
            top =  np.random.randint(0, img_h-h-1,size=n)
            for j in range(n):
              x[i,top[j]:top[j] + h, left[j]:left[j] + w, :] = 0

        return x,y

In [0]:
def ps_generator_fs(gen,n=4, p=0.5, s_l=0.05,s_h=0.1,r_1=0.8,r_2=1.2,st=STATS_FS):
  while True:
    batch = next(gen)
    x_res,y_res = ps_batch(batch,n=n,p=p,s_l=s_l,s_h=s_h,r_1=r_1,r_2=r_2,st=st)
    yield x_res,y_res

In [0]:
def cutmix_generator_fs(gen1,gen2,alpha=0.4,h=HEIGHT,w=WIDTH,st=STATS_FS):
    while True:
        batch1 = next(gen1)
        batch2 = next(gen2)
        x_res,y_res = cutmix_batches_fs(batch1,batch2,alpha=alpha,h=h,w=w,st=st)

        yield x_res,y_res

In [0]:
def mixup_generator_fs(gen1,gen2,alpha=0.4,h=HEIGHT,w=WIDTH,st=STATS_FS):
    while True:
        batch1 = next(gen1)
        batch2 = next(gen2)
        x_res,y_res = mix_batches_fs(batch1,batch2,alpha=alpha,st=st)

        yield x_res,y_res

In [0]:
def cutout_generator_fs(gen,st=STATS_FS):
  while True:
    batch = next(gen)
    x_res,y_res = prep_batch_fs(batch,st=st)
    yield x_res,y_res

In [0]:
def get_aug_methods(ps_n=4,ps_p=0.5,cm_alpha=0.4,mx_alpha=0.4,st=STATS_FS):
  aug_methods={}
  aug_methods[f'prog_spr_{ps_n}_{ps_p:.2f}']=ps_generator_fs(mcm_generator3, n=ps_n, p=ps_p, st=st)
  aug_methods[f'cutmix_{cm_alpha:.2f}']=cutmix_generator_fs(mcm_generator1,mcm_generator2,alpha=cm_alpha, st=st)
  aug_methods[f'mixup_{mx_alpha:.2f}']=mixup_generator_fs(mcm_generator1,mcm_generator2,alpha=mx_alpha,h=HEIGHT,w=WIDTH,st=st)
  aug_methods['cutout']=cutout_generator_fs(cutout_generator,st=st)

  return aug_methods

In [0]:
if 0: 
  batch_x,y = next(cutout_datagen_fs)
  import matplotlib.pyplot as plt
  fig, axs = plt.subplots(1, 7, figsize=(30, 30))
  for i in range(7):
    axs[i].imshow(batch_x[i,:,:,0])
  plt.show()

In [0]:
if 0: opt = Lookahead(tf.keras.optimizers.SGD(learning_rate=LR_MAX, momentum=0.9,nesterov=True))

opt=tf.keras.optimizers.Adam(learning_rate=LR_MAX)

In [0]:
checkpoint =  tf.train.Checkpoint(latest_epoch=tf.Variable(0), optimizer=opt, model=model)
chkp_manager =  tf.train.CheckpointManager(checkpoint, CHKP_DIR, max_to_keep=3)

In [0]:
chkp_manager.latest_checkpoint

'/content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_augrot/chkp/ckpt-126'

In [0]:
chkp_restore_status=checkpoint.restore(chkp_manager.latest_checkpoint)
if chkp_manager.latest_checkpoint:
  #chkp_restore_status.assert_consumed()
  print("Restored from {}".format(chkp_manager.latest_checkpoint))
  print(f"Lastest epoch {checkpoint.latest_epoch.numpy()}")
else:
  print("Initializing from scratch.")

Restored from /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_augrot/chkp/ckpt-126
Lastest epoch 126


In [0]:
loss_dict={'root':    'categorical_crossentropy',
           'vowel':    'categorical_crossentropy',
           'consonant':'categorical_crossentropy'}
model.compile(optimizer=opt, loss=loss_dict, loss_weights=[2.0,1.0,1.0])

In [0]:
initial_epoch = checkpoint.latest_epoch.numpy()

In [0]:
#ca_shed = CosineAnnealingScheduler(T_max=T_MAX, eta_max=LR_MAX, eta_min=LR_MIN,verbose=1, initial_epoch=initial_epoch-130)
#rolrplt = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=15, verbose=1, factor=0.5, min_lr=LR_MIN)
CYCLES = 10
LR_FACTOR=0.95
best_kaggle = 0.9840
cur_lr = tf.keras.backend.get_value(model.optimizer.lr)


for cycle in range(CYCLES):
    ps_n = random.randint(4,16)
    ps_p = random.uniform(0.2,0.8)
    cm_alpha = random.uniform(0.4,1.0)
    mx_alpha = random.uniform(0.1,0.4)
    print(f'cycle:{cycle} parameters: ps_n:{ps_n} - ps_p:{ps_p:.4f} - cm_a:{cm_alpha:.4f} - mx_a:{mx_alpha:.4f}')
    print(f'best kaggle:{best_kaggle}')
    aug_methods = get_aug_methods(ps_n=ps_n,ps_p=ps_p,cm_alpha=cm_alpha,mx_alpha=mx_alpha)
    method_keys = list(aug_methods.keys())
    random.shuffle(method_keys)
    best_cycle_kaggle_values=[]
    print(f'seting lr to {cur_lr:.5f} for cycle: {cycle}')
    tf.keras.backend.set_value(model.optimizer.lr, cur_lr)
    for method in method_keys:
      method_epochs = np.random.randint(2,6)
      print(f'augmentation method: {method} - cycle epochs: {method_epochs}')
  
      suffix =EXPERIMENT+\
                '_aug_'+method+\
                '_meps'+str(method_epochs)+\
                '_eps'+str(EPOCHS)+\
                '_iniep'+str(initial_epoch)+\
                '_sz'+str(IMG_SIZE)+\
                '_bs'+str(BATCH_SIZE)+\
                '_do'+str(DROP_OUT)+\
                '_'+model.optimizer.get_config()['name']+\
                '_IS_fold'+str(FOLD)

      kv = KaggleValidationFS(valid_df,
                      batch_size=512,
                      vals_per_epoch=2,
                      train_steps=train_m//BATCH_SIZE+1,
                      logfile=EXPERIMENT_DIR+'log_kr_'+suffix+'.csv',
                      initial_epoch=initial_epoch,
                      suffix = suffix,
                      train_img_dir=TRAIN_IMG_DIR,
                      output_dir=EXPERIMENT_DIR,
                      thresholds=[0.978,0.995,0.993],
                      chkp_manager=chkp_manager,
                      in_shape=[HEIGHT,WIDTH],
                      st=STATS_FS)

      cbs = [kv]

      print(f'epochs trained: {initial_epoch}. start training epoch {initial_epoch+1}...')
      history=model.fit(aug_methods[method],
                  epochs=method_epochs,
                  steps_per_epoch=train_m//BATCH_SIZE+1,
                  callbacks=cbs,
                  class_weight=model_cw,
                  verbose=2)
      
      initial_epoch = initial_epoch + method_epochs
      best_method_kaggle = max(kv.val_kaggle_recalls)
      print(f"best kaggle: {best_method_kaggle:.4f} in method:{method} and cycle: {cycle}")
      best_cycle_kaggle_values.append(best_method_kaggle)

    best_cycle_kaggle = max(best_cycle_kaggle_values)
    print(f'best kaggle metric in cycle: {best_cycle_kaggle:.4f}')

    if(best_cycle_kaggle>best_kaggle):
      best_kaggle = best_cycle_kaggle
    else:
      cur_lr = cur_lr*LR_FACTOR
  


#model.save_weights(EXPERIMENT_DIR+'w_final_checkp_'+suffix+'.h5')
#model.save(OUTPUT_DIR+'m_'+suffix+'.h5')
#pd.DataFrame(history.history).to_csv(OUTPUT_DIR+'h_'+suffix+'.csv', index=False)

cycle:0 parameters: ps_n:4 - ps_p:0.2630 - cm_a:0.4166 - mx_a:0.1683
best kaggle:0.984
seting lr to 0.00100 for cycle: 0
augmentation method: prog_spr_4_0.26 - cycle epochs: 5
epochs trained: 138. start training epoch 139...
Train for 1491 steps
Epoch 1/5

batch 744: validation...



[0:11:38] - kaggle:0.9830 - root:0.9756 -vowel:0.9917 - cons: 0.9892

batch 1489: validation...



[0:23:17] - kaggle:0.9827 - root:0.9760 -vowel:0.9910 - cons: 0.9879

saved checkpoint for epoch 139: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_augrot/chkp/ckpt-139

1491/1491 - 1401s - loss: 0.1024 - root_loss: 0.0076 - vowel_loss: 0.0070 - consonant_loss: 0.0041
Epoch 2/5

batch 744: validation...



[0:35:00] - kaggle:0.9840 - root:0.9768 -vowel:0.9917 - cons: 0.9908

batch 1489: validation...



[0:46:38] - kaggle:0.9834 - root:0.9782 -vowel:0.9890 - cons: 0.9884
saving weights w_best_[r 0.97821] best combined kaggle recall: 0.9861053982481158...

saved checkpoint for epoch 140: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_augrot/chkp/ckpt-140

1491/1491 - 1402s - loss: 0.1010 - root_loss: 0.0602 - vowel_loss: 0.0224 - consonant_loss: 0.0352
Epoch 3/5

batch 744: validation...


...


[2:04:47] - kaggle:0.9812 - root:0.9721 -vowel:0.9912 - cons: 0.9893

saved checkpoint for epoch 169: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_augrot/chkp/ckpt-169

1491/1491 - 1876s - loss: 1.2910 - root_loss: 0.2900 - vowel_loss: 0.2608 - consonant_loss: 0.1834
best kaggle: 0.9826 in method:mixup_0.16 and cycle: 2
augmentation method: cutmix_0.95 - cycle epochs: 5
epochs trained: 169. start training epoch 170...
Train for 1491 steps
Epoch 1/5

batch 744: validation...



[0:15:23] - kaggle:0.9800 - root:0.9716 -vowel:0.9889 - cons: 0.9879

batch 1489: validation...



[0:30:45] - kaggle:0.9801 - root:0.9715 -vowel:0.9881 - cons: 0.9892

saved checkpoint for epoch 170: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_augrot/chkp/ckpt-170

1491/1491 - 1850s - loss: 5.2740 - root_loss: 1.4004 - vowel_loss: 0.7974 - consonant_loss: 1.7330
Epoch 2/5

batch 744: validation...



[0:46:04] - kaggle:0.9804 - root:0.9715 -vowel:0.9901 - cons: 0.9886

batch 1489: validation...



[1:00:49] - kaggle:0.9788 - root:0.9710 -vowel:0.9881 - cons: 0.9849

saved checkpoint for epoch 171: /content/drive/My Drive/kaggle/bengali/output/sernext_3cmc_fullsize_augrot/chkp/ckpt-171

1491/1491 - 1803s - loss: 5.1225 - root_loss: 1.2383 - vowel_loss: 0.8650 - consonant_loss: 0.5549
Epoch 3/5


In [0]:
model.optimizer.get_config()